In [2]:
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import load_tr_cities, translation
warnings.filterwarnings('ignore')
path = '../../../datasets/garanti-bbva-data-camp/education.csv'

In [16]:
def load_education(path: str, study_size: int = 10, degree_size: int = 10) -> pd.DataFrame:

    df_ = pd.read_csv(path)

    # Fields of Study
    df_.loc[df_['fields_of_study'] == 'Bilgisayar Mühendisliği', 'fields_of_study'] = 'Computer Engineering'
    df_.loc[df_['fields_of_study'] == 'Elektrik ve Elektronik Mühendisliği', 'fields_of_study'] = 'Electrical and Electronics Engineering'
    df_.loc[df_['fields_of_study'] == 'Bilgisayar Programlama', 'fields_of_study'] = 'Computer Programming'
    df_.loc[df_['fields_of_study'] == 'bilgisayar programcılığı', 'fields_of_study'] = 'Computer Programming'
    df_.loc[df_['fields_of_study'] == 'Bilgisayar Programcılığı', 'fields_of_study'] = 'Computer Programming'
    df_.loc[df_['fields_of_study'] == 'bilgilsayar programcılığı', 'fields_of_study'] = 'Computer Programming'
    df_.loc[df_['fields_of_study'] == 'Matematik', 'fields_of_study'] = 'Mathematics'
    df_.loc[df_['fields_of_study'] == 'İşletme ve Yönetim, Genel', 'fields_of_study'] = 'Business Administration and Management, General'
    df_.loc[df_['fields_of_study'] == 'Bilgisayar Yazılımı Mühendisliği', 'fields_of_study'] = 'Software Engineering'
    df_.loc[df_['fields_of_study'] == 'Yazılım Mühendisliği', 'fields_of_study'] = 'Software Engineering'
    df_.loc[df_['fields_of_study'] == 'Endüstri Mühendisliği', 'fields_of_study'] = 'Industrial Engineering'
    df_.loc[df_['fields_of_study'] == 'Makine Mühendisliği', 'fields_of_study'] = 'Mechanical Engineering'
    df_.loc[df_['fields_of_study'] == 'Fizik', 'fields_of_study'] = 'Physics'
    df_.loc[df_['fields_of_study'] == 'Ekonomi', 'fields_of_study'] = 'Economics'
    df_.loc[df_['fields_of_study'] == 'İstatistik', 'fields_of_study'] = 'Statistics'
    df_.loc[df_['fields_of_study'] == 'Kimya', 'fields_of_study'] = 'Chemistry'
    df_.loc[df_['fields_of_study'] == 'Elektrik Mühendisliği', 'fields_of_study'] = 'Electrical Engineering'
    df_.loc[df_['fields_of_study'] == 'Elektrik mühendisliği', 'fields_of_study'] = 'Electrical Engineering'
    df_.loc[df_['fields_of_study'] == 'Elektrik Mühendisliği', 'fields_of_study'] = 'Electrical Engineering'
    df_.loc[df_['fields_of_study'] == 'Elektrik Mühendisi', 'fields_of_study'] = 'Electrical Engineering'
    df_.loc[df_['fields_of_study'] == 'elektrik mühendisliği', 'fields_of_study'] = 'Electrical Engineering'
    df_.loc[df_['fields_of_study'] == 'bilgisayar mühendisliği', 'fields_of_study'] = 'Computer Engineering'
    df_.loc[df_['fields_of_study'] == 'bilgisayar mühendisliği ', 'fields_of_study'] = 'Computer Engineering'
    df_.loc[df_['fields_of_study'] == 'bilgisayar mühendisligi', 'fields_of_study'] = 'Computer Engineering'
    df_.loc[df_['fields_of_study'] == 'bilgisayar müh.', 'fields_of_study'] = 'Computer Engineering'
    df_.loc[df_['fields_of_study'] == 'bilgisayar mühendisi', 'fields_of_study'] = 'Computer Engineering'
    df_.loc[df_['fields_of_study'] == 'yönetim bilişim sistemleri', 'fields_of_study'] = 'Management Information Systems'
    df_.loc[df_['fields_of_study'] == 'Yönetim Bilişim Sistemleri', 'fields_of_study'] = 'Management Information Systems'


    ################################################################################################################

    # Degree
    high_school = 'High School'
    bachelors_deg = "Bachelor's degree"
    associates_deg = "Associate's degree"
    masters_deg = "Master's degree"
    bsc = 'Bachelor of Science'
    msc = 'Master of Science'
    mba = 'Master of Business Administration'
    bba = 'Bachelor of Business Administration'
    phd = 'Doctor of Philosophy'
    basc = 'Bachelor of Applied Science'
    be = 'Bachelor of Engineering'
    me = 'Master of Engineering'


    #Bachelors degree
    df_.loc[df_['degree'] == 'Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (2,68/4)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor's Degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor’s Degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "bachelor", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor Degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "bachelor's degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "bachelor's", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "bachelor degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == " bachelor's degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Electrical and Electronics Engineering bachelor degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "bachelor degree 3.07", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Three years of informatics completed successfully out of 5 years bachelor degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Bachelor degree 2,79/4', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Lisans Derecesi / Bachelor's degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (2.90/4.00 - 4.00/4.00)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi ', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi  ', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi 3.35\\4.00', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == '3.16/4 Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == '2.72/4 Çift Anadal / Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == ' 3.35 Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi(Erasmus)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi Mezunu', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Lisans Derecesi,Bachelor's Degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi, İngilizce', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi, Çift Anadal', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi Fakülte Birinciliği ', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi, 3.45', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Çift Anadal Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi / License Degree', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi, 3.57/4.00', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (Bölüm Birincisi) ', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (Bachelor)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi- Dokuz Eylül Üniversitesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi 3.3/4', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi , 3.49', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi: 2.90', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Bachelors / Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi  GPA : 3.36/4', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi - 3,32 / 4', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi 2.83/4', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi %25 Burslu', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == '2.91/4-Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (terk) / Undergraduate (dropped)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi: (TERK)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi / Bachelor Degree', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (Bachelor’s Degree)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi, 3.32', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor's degree, Bachelor of Science - BS, Lisans Derecesi", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Bilgisayar Mühendisliği Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (Erasmus)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'YG, Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi - Bachelor Degree', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi-  3.76/4 ', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == '3.30 Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi(4.)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == '3,21 - Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi (3.36)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi - Erasmus', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi, Bölüm Birincisi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == '3.52 - Lisans Derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi ( Mühendislik Fakültesi )', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi,Hazırlık(İng)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Derecesi:2.54', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans ', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans, Anadal', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Lisans / Bachelor's degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans, İkinci Anadal', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans ( Açık Öğretim)', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Mezunu', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Öğrencisi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans Açık Öğretim', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor's degree / Lisans", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == 'Lisans derecesi', 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor's degree / Lisans ", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "lisans", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor degree", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelor's", 'degree'] = bachelors_deg
    df_.loc[df_['degree'] == "Bachelors", 'degree'] = bachelors_deg
    
    
    #Associates degree
    df_.loc[df_['degree'] == "Önlisans", 'degree'] = associates_deg
    df_.loc[df_['degree'] == "Ön Lisans", 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'önlisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'ön lisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'ön lisans ', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'önlisans ', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'ön Lisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'ön lisana', 'degree'] = associates_deg
    df_.loc[df_['degree'] == ' ön lisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'ÖnLisans Derecesi', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'Ön Lisans Derecesi', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'On Lisans Derecesi', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'ÖnLisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'Ön Lisans ', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'Açık Öğretim / Ön Lisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'Ön Lisans (derece ile)', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'Ön lisans', 'degree'] = associates_deg
    df_.loc[df_['degree'] == 'Associate Degree', 'degree'] = associates_deg
    df_.loc[df_['degree'] == "Associate’s Degree", 'degree'] = associates_deg
    df_.loc[df_['degree'] == "Associate's Degree", 'degree'] = associates_deg
    df_.loc[df_['degree'] == "Associate", 'degree'] = associates_deg

    
    #Masters degree
    df_.loc[df_['degree'] == 'Tezli Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Yüksek Lisans (Master)", 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Yüksek Lisans", 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Master’s Degree", 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Master's Degree", 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Master", 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans ', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master-3,25)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Fen Edebiyat Fakültesi - Yüksek Lisans - Tez Safhası ( Devam Etmekte )', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) Özel Öğrenci', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans Fen Bilimleri Enstitüsü', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master Degree)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Y.Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksel Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yuksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Tezli Yüksek Lisans (Master)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master), Bilgisayar Mühendisliği', 'degree'] = masters_deg
    df_.loc[df_['degree'] == '(Yüksek Lisans)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'İstatistik Bölümü Yüksek Lisans Mezunu', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) 3.90', 'degree'] = masters_deg    
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master), İngilizce', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (MSc)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Mühendislik ve Fen Bilimleri Enstitüsü, Bilgisayar Mühendisliği Yüksek Lisans (İngilizce)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) ', 'degree'] = masters_deg
    df_.loc[df_['degree'] == '3.56 Yüksek Lisans (Master)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Tezli Yüksek Lisans-MSc Degree', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Ms)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans - Fen Bilimleri Enstitüsü', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) Elektronik ve Haberleşme Mühendisliği', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Fen Bilimleri Enstitüsü (Yüksek Lisans)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Tezsiz Yüksek Lisans Özel Öğrenci', 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Master's degree / Yüksek Lisans", 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Mühendislik Fakültesi - Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Fen Bilimleri Enstitütüsü / Tezsiz Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Lisans ile Birleştirilmiş Tezsiz Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) Tam Burslu', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Tezsiz Yüksek Lisans (Master)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Gazi Üniversitesi Fizik Yüksek Lisans Mezunu', 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Yüksek Lisans / Master's degree", 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) (Tezli)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Pazarlama Yüksek Lisans Öğrencisi', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Fen Bilimleri Fakültesi - Yüksek Lisans (Tezli)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans Öğrenci', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master’s degree)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans Tezli (Master)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Fen Bilimleri Enstitüsü Elektronik ve Bilgisayar Eğitimi Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == "Yüksek Lisans / Master's Degree", 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Elektronik ve Haberleşme Mühendisliği Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Fen Bilimleri (Yüksek Lisans)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'İşletme Yüksek Lisans Mezun', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Enformatik Tezli Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Tezli Yüksek Lisans (Master) ', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) - dropped', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master)-Tezli', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans - Tezsiz -', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Tezsiz) ', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master) Öğrencisi', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Bilgisayar Mühendisliği Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Tezsiz Yüksek Lisans', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans Öğrencisi', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'Yüksek Lisans (Master)(REMEDIAL)', 'degree'] = masters_deg
    df_.loc[df_['degree'] == 'yüksek lisans', 'degree'] = masters_deg


    #High school
    df_.loc[df_['degree'] == "Lise", 'degree'] = high_school
    df_.loc[df_['degree'] == "High School Diploma", 'degree'] = high_school
    df_.loc[df_['degree'] == 'lise', 'degree'] = high_school
    df_.loc[df_['degree'] == 'lise ', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Anadolu lisesi mezunu', 'degree'] = high_school
    df_.loc[df_['degree'] == 'süper lise', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Endüstri lise', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Teknik lise', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Ortaöğretim(lise)', 'degree'] = high_school
    df_.loc[df_['degree'] == 'lise mezunu', 'degree'] = high_school
    df_.loc[df_['degree'] == 'imkb anadolu teknik lisesi', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Meslek lisesi', 'degree'] = high_school
    df_.loc[df_['degree'] == 'lise/ High School ', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Anadolu meslek lisesi mezunu', 'degree'] = high_school
    df_.loc[df_['degree'] == 'Fen lisesi mezunu', 'degree'] = high_school
    df_.loc[df_['degree'] == 'izmit lisesi', 'degree'] = high_school
    df_.loc[df_['degree'] == '700. yıl anadolu lisesi', 'degree'] = high_school
    df_.loc[df_['degree'] == 'LİSE', 'degree'] = high_school
    df_.loc[df_['degree'] == 'ANADOLU TEKNİK LİSESİ', 'degree'] = high_school
    df_.loc[df_['degree'] == 'SABANCI ANADOLU TEKNİK LİSESİ', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Kabataş Erkek Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Hınıs imam hatip lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Atatürk lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Malatya lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'İzmir Atatürk Lisesi'] = high_school
    df_.loc[df_['school_name'] == 'Ankara Atatürk Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Bornova Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Sırrı Yırcalı Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Kadıköy Anadolu Lisesi ve Maarif Koleji', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Bornova Anadolu Lisesi (İzmir Koleji)', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Beşiktaş Atatürk Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Ankara Atatürk Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Istanbul Erkek Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Burak Bora Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Haydarpaşa Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Karşıyaka Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Kadıköy Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Bursa Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Denizli Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'İstanbul Köy Hizmetleri Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Şehremini Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Tekirdağ Fen Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'İstanbul Atatürk Fen Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Bahçelievler Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Kocaeli Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Adnan Menderes Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Hüseyin Avni Sözen Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Tekirdağ Anadolu Lisesi', 'degree'] = high_school
    df_.loc[df_['school_name'] == 'Gazi Anadolu Lisesi', 'degree'] = high_school


    #BSc
    df_.loc[df_['degree'] == 'Bachelor of Science - BS', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BS)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (B.Sc.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (B.S.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - BSc', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science(B.Sc.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Double Major, Bachelor of Science - BSc', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (B.Sc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science Degree', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - BSc.', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc), 2nd Degree of Graduation', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc),', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - B.Sc', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - BSc (Hons)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (Hons.B.Sc.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Double Major, Bachelor of Science (B.S.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BS), Mechanical Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science ', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - (BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science in Economics- BSc(Econ)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - BS, Physics, 3.06/4.00', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - B.Sc.', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science(BS)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science Computer Science - BS(CS)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - B.S.', 'degree'] = bsc
    df_.loc[df_['degree'] == ' Bachelor of Science', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BS) ', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) - Not graduate', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc: Bachelor of Science', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (M.S.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science in Computer Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science(B.Sc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science in Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BS), Chemistry', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BS), Electrical and Electronics Engineering, GPA 4.00', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science - BS, Computer Technologies and Information Systems', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Sciences - BS', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science With Honours', 'degree'] = bsc
    df_.loc[df_['degree'] == 'bachelor of science', 'degree'] = bsc
    df_.loc[df_['degree'] == "Bachelor's of science", 'degree'] = bsc
    df_.loc[df_['degree'] == "BS", 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc.', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc Engineering ', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc., Electrical Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == "Bachelor's Degree (BSc)", 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc in Computer Engineering 3.66/4.00', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Lisans Derecesi(BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == "Bachelor's degree (BSc)", 'degree'] = bsc
    df_.loc[df_['degree'] == 'Electrical and Electronics Engineering (BSc.)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science(BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc. Honours', 'degree'] = bsc
    df_.loc[df_['degree'] == '3.22/4, Lisans (BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) - Computer Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science  (BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc-Eng)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc, Jeodesy and Photogrameteri', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) ( Transferred to Middle East Technical University after 1st year)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc(Double Major)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science - BSc', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc,', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Mechanical Engineering (BSc) 2005', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) (3.16 honored student)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc. Department of Electronic', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) (3.06 honored student)', 'degree'] = bsc
    df_.loc[df_['degree'] == "Bachelor's degree, BSc.", 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc. Degree', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BS / BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science(BSc), Computer Enginnering', 'degree'] = bsc
    df_.loc[df_['degree'] == "Bachelor's of Science (BSc)", 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc (Hons) in Software Design with Cloud Computing', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc in Computer Science, 3.48/4.00 (CGPA)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc. In Dept. of Computer Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Undergraduate (BSc) with honors', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science (BSc)', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) - Department of Information Technologies', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc), Geology/Earth Science', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science(BSc), Computer Science', 'degree'] = bsc
    df_.loc[df_['degree'] == 'Bachelor of Science (BSc) with Honors', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc Degree', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc. Electronics&Communication Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSc, Computer Engineering', 'degree'] = bsc
    df_.loc[df_['degree'] == 'BSC','degree'] = bsc
    df_.loc[df_['degree'] == 'BSCS(HONOURS)','degree'] = bsc
    df_.loc[df_['degree'] == 'B.Sc.','degree'] = bsc
    df_.loc[df_['degree'] == 'B.S.','degree'] = bsc
    df_.loc[df_['degree'] == 'B.S','degree'] = bsc

    
    #MSc
    df_.loc[df_['degree'] == 'Master of Science (MSc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MS', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MS)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.S.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MSc', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science ', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc), 1st Degree of Graduation', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - M.Sc.', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science Degree', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science(M.Sc.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc) ', 'degree'] = msc
    df_.loc[df_['degree'] == 'Research Assistant - Master of Science (MSc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in IT (M.I.T.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MS, Physics, 3.61/4.00', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science MSc', 'degree'] = msc
    df_.loc[df_['degree'] == 'Msc Master of Science in Electrical& Electronics Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc), Computer Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - Informatik', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MSc.', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - M.Sc', 'degree'] = msc
    df_.loc[df_['degree'] == ' Master of Science - MS ', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Information Technology', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science(MSc), Computer Software Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - Ms', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Engineering Management - MSEM', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science- MSc', 'degree'] = msc
    df_.loc[df_['degree'] == '3.44/4.00 Master of Science (MSc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M. Sc.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Engineering - MScEng', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - M. Sc.', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc), Mak. Dinamiği, Titreşim&Akustiği', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc.) in Electronics Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Yüksek Lisans - Master of Science (MSc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science  (M.Sc.)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science(MSc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Information Systems (MSIS)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science / Yüksek Lisans', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MS(without thesis)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science -MS', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MSc Student', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - M.Sc. , 3.83/4', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Informatics Institute   ', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc.) - DROPPED', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - M.S', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MS) with Thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc)-Solid Mechanics', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science(MS)', 'degree'] = msc
    df_.loc[df_['degree'] == 'M.Sc. - Master of Science', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MS (with thesis) ', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MS,Computer Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Bachelor and Master of Science-MS,Chemistry(German)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc) with Thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc.) ', 'degree'] = msc
    df_.loc[df_['degree'] == '3.88 / 4 - Master of Science (MS), Applied Informatics', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc.) Mechanical Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science, CGPA: 3.86/4.00', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Computer Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Computer Science (MSCS)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc) without thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc-dropped)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc) - dropped due to financial reasons', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science  (M.Sc.) with Thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science and Engineering', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science - MSc with Thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (M.Sc.) Biology', 'degree'] = msc
    df_.loc[df_['degree'] == 'Yüksek Lisans ( M.Sc.), Graduate, Master of Science', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc), Geology/Earth Science', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc)  (UK)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science  (MSc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science in Engineering (M.Sc.Eng)', 'degree'] = msc
    df_.loc[df_['degree'] == 'Master of Science (MSc) - Non Thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'MSC - Thesis', 'degree'] = msc
    df_.loc[df_['degree'] == 'MSC', 'degree'] = msc
    df_.loc[df_['degree'] == 'MSC.', 'degree'] = msc
    df_.loc[df_['degree'] == 'MSc', 'degree'] = msc
    df_.loc[df_['degree'] == 'Yüksek Lisans (M.Sc)', 'degree'] = msc
    df_.loc[df_['degree'] == 'MS', 'degree'] = msc
    df_.loc[df_['degree'] == 'M.Sc.', 'degree'] = msc
    df_.loc[df_['degree'] == 'M.S.', 'degree'] = msc


    #MBA
    df_.loc[df_['degree'] == 'Master of Business Administration - MBA', 'degree'] = mba
    df_.loc[df_['degree'] == 'Master of Business Administration (MBA)', 'degree'] = mba
    df_.loc[df_['degree'] == 'Master of Business Administration (M.B.A.)', 'degree'] = mba
    df_.loc[df_['degree'] == 'MBA', 'degree'] = mba
    df_.loc[df_['degree'] == 'Mba', 'degree'] = mba
    df_.loc[df_['degree'] == '(MBA)-Marketing', 'degree'] = mba
    df_.loc[df_['degree'] == 'Yüksek Lisans (MBA)', 'degree'] = mba
    df_.loc[df_['degree'] == 'MBA,Master of Business Administration', 'degree'] = mba
    df_.loc[df_['degree'] == '(MBA)', 'degree'] = mba
    df_.loc[df_['degree'] == "MBA 3.8", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Management - (MBA)", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Administration, MBA", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Administration - MBA ", 'degree'] = mba
    df_.loc[df_['degree'] == "MBA (Master Degree) ", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Administration (MBA) - English", 'degree'] = mba
    df_.loc[df_['degree'] == "MBA degree", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Administration   3.87/4.00 GPA", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Administration (M. B. A.)", 'degree'] = mba
    df_.loc[df_['degree'] == "The Master of Business Administration (MBA)", 'degree'] = mba
    df_.loc[df_['degree'] == "Master of Business Administration -3,75", 'degree'] = mba
    df_.loc[df_['degree'] == 'İşletme Yüksek Lisans Programı (MBA)', 'degree'] = mba
    df_.loc[df_['degree'] == 'İşletme Tezsiz Yüksek Lisans Programı (MBA)', 'degree'] = mba
    df_.loc[df_['degree'] == 'İşletme Yüksek Lisans Programı / MBA', 'degree'] = mba
    df_.loc[df_['degree'] == 'GPA: 4.00/4.00 ,Master of Business Administration (M.B.A.)', 'degree'] = mba


    #BASc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science (B.A.Sc.)', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science - BASc', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science (BASc)', 'degree'] = basc
    df_.loc[df_['degree'] == 'Erasmus Student at Bachelor of Applied Science - BASc', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science (B.A.Sc.), Physics Engineering', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Sciences - BAS', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science - (BASc) ', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science - B.A.Sc.', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science (B.A.Sc', 'degree'] = basc
    df_.loc[df_['degree'] == 'Bachelor of Applied Science (B.Sc.)', 'degree'] = basc


    #BE
    df_.loc[df_['degree'] == 'Bachelor of Engineering - BE', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.E.)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.Eng.)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (BEng)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (BE)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B. E.)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering(BE)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering - BE (Honoured)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering - BE ', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.Eng)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (Eng.)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering - BE Outstanding Scholarship (%100)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering(Minor Program)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.E.), Computer Engineering', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.Eng.), High Honor List', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering - BEng', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.Eng.) Honour Degree', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.Sc.)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering - Computer Engineering', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (BE), Computer Science', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (B.E.), Makine Mühendisliği', 'degree'] = be
    df_.loc[df_['degree'] == "Bachelor of Engineering (B.Eng.)ineer's degree", 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering (%100 B.Eng.)', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering - B.Eng.', 'degree'] = be
    df_.loc[df_['degree'] == 'Bachelor of Engineering ', 'degree'] = be
    df_.loc[df_['degree'] == 'bachelor of engineering', 'degree'] = be
    df_.loc[df_['degree'] == "Engineer's degree", 'degree'] = be
    df_.loc[df_['degree'] == "Engineer's degree, Mechanical Engineering", 'degree'] = be
    df_.loc[df_['degree'] == "Engineer's degree,", 'degree'] = be
    df_.loc[df_['degree'] == "Engineer's degree (Exchange Student)", 'degree'] = be

    
    #ME
    df_.loc[df_['degree'] == 'Master of Engineering - MEng', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (M.Eng.)', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (MEng)', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (M. E.)', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering Management', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (M.Eng.), Thermodynamics', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (MSc)', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (M.Eng.) Student', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering - Mechanical', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering (M.Eng.), Makine Mühendisliği', 'degree'] = me
    df_.loc[df_['degree'] == 'Master of Engineering and Business (M.Eng.)', 'degree'] = me

    
    #BBA
    df_.loc[df_['degree'] == 'Bachelor of Business Administration - BBA', 'degree'] = bba
    df_.loc[df_['degree'] == 'Bachelor of Business Administration (B.B.A.)', 'degree'] = bba
    df_.loc[df_['degree'] == 'Bachelor of Business Administration (BBA)', 'degree'] = bba
    df_.loc[df_['degree'] == 'Bachelor of Business Administration', 'degree'] = bba
    df_.loc[df_['degree'] == 'Bachelor of Business Administration - BBA ', 'degree'] = bba
    df_.loc[df_['degree'] == 'Bachelor of Bussines Administration', 'degree'] = bba
    df_.loc[df_['degree'] == 'Bachelor of Business Administration - BA', 'degree'] = bba
    df_.loc[df_['degree'] == 'BA', 'degree'] = bba


    #PhD
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (PhD)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD(C)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD Candidate', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD ', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD (Candidate)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD cand.', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (PhD) (Candidate)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.) - Abandoned', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD(Integrated)', 'degree'] = phd
    df_.loc[df_['degree'] == '•\tUniversiad Azteca Doctor of Philosophy in Business Administration , PhD (ID : 161AS19D0316)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (PhDc)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (PhD) (Dropped Out)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.) in Business Administration', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (PhD.) Candidate', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D., Dr.-Ing.)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.) (Not finished)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy(PhD)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD candidate', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (PhD) Student ', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.) (not completed)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy Canditate- PhDc', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph. D.)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD (Working)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD (leave)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.) (c)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy - PhD  (Dropout)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D. Candidate)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.)(leave of absence)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (Ph.D.) - Dropped out of', 'degree'] = phd
    df_.loc[df_['degree'] == "Doctor of Philosophy - PhD (Cont'd)", 'degree'] = phd
    df_.loc[df_['degree'] == 'Drop Out - Doctor of Philosophy (PhD) ', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doctor of Philosophy (MSc & PhD Integrated) ', 'degree'] = phd
    df_.loc[df_['degree'] == 'PhD', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doktora ( PHD )', 'degree'] = phd
    df_.loc[df_['degree'] == 'Doktora (Dr.)', 'degree'] = phd
    df_.loc[df_['degree'] == 'Ph.D.', 'degree'] = phd
    df_.loc[df_['degree'] == 'phd', 'degree'] = phd

    
    #Undefined
    df_.loc[df_['degree'] == '3.65/4.00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '52.95', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.5', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.50', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.01', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,5', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.14', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,7', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.30', 'degree'] = np.nan
    df_.loc[df_['degree'] == '73,25', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.2', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,14', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.05', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.0', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.20', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.15', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.10', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.3', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.02', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.65', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.22', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.06', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.55', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.16', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.40', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,12', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.68', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.27', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,25', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.1', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.75', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.28', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,20', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.03', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.33', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.54', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.21', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.23', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,50', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,2', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.09', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,78', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.13', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.17', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.66', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.47', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.11', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.24', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.07', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,21', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2.83', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.80', 'degree'] = np.nan
    df_.loc[df_['degree'] == '4.63', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,05', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2.63', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.26', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,13', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,86', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.08', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3,62', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.52', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.5/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '93', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.04', 'degree'] = np.nan
    df_.loc[df_['degree'] == '83', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2.73', 'degree'] = np.nan
    df_.loc[df_['degree'] == '3.70', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,80', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,5', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,7/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,75', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,87', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,42', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,77', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,50', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,63', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,66', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,82', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,92', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,7', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,71', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,48', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,65', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,98/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,95', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,76 / 4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,5/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,84', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,97', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,67', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,85', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,60', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,34', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,74', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,63/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,58', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,61', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,62', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,51', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,59', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,47', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,8', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,99', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,79', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,29', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,89', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,57', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,72', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,91', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,53', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,52', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,6', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,73', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,0', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,47 / 4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,35', 'degree'] = np.nan
    df_.loc[df_['degree'] == '92,2 /100', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,54 / 4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,90/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,36/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,54', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,36', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,86', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,70', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,58 / 4,0', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,55', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,30', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,64', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,44', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,96', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,99 / 4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,93', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,76/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,76', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,9', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,38', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,98', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,12', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,19', 'degree'] = np.nan
    df_.loc[df_['degree'] == '92,04', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,43', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,37', 'degree'] = np.nan
    df_.loc[df_['degree'] == '92,6 / 100', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,76/4.00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,2', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,80/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,41', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,88', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,56', 'degree'] = np.nan
    df_.loc[df_['degree'] == '72,28/100', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,22', 'degree'] = np.nan
    df_.loc[df_['degree'] == '4 / 2,92', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,69/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,85/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,90', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,67/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,85 / 4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,44 / 4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,68', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,78', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,25', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,24 /4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,02/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,58/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,35/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '82,29', 'degree'] = np.nan
    df_.loc[df_['degree'] == '92,07', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,5 / 4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,28', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,75/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,70/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,3/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,86/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '92,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '72,65', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,81', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,90/4', 'degree'] = np.nan
    df_.loc[df_['degree'] == '82,5', 'degree'] = np.nan
    df_.loc[df_['degree'] == '92,25', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,3', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,94', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,39', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,47/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,52/4,00', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,69', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,15', 'degree'] = np.nan
    df_.loc[df_['degree'] == '2,83', 'degree'] = np.nan

    df_ = df_.drop_duplicates()
    most_freq_studies = df_['fields_of_study'].value_counts().keys()[:study_size].tolist()
    most_freq_degrees = df_['degree'].value_counts().keys()[:degree_size].tolist()

    study_grouped = df_.groupby(by='user_id', as_index=False).agg({'fields_of_study': 'unique'})
    degree_grouped = df_.groupby(by='user_id', as_index=False).agg({'degree': 'unique'})

    for study in tqdm(most_freq_studies):
        study_grouped[f'study_{study}'] = study_grouped['fields_of_study'].apply(lambda x: 1 if study in x else 0)

    for degree in tqdm(most_freq_degrees):
        degree_grouped[f'degree_{degree}'] = degree_grouped['degree'].apply(lambda x: 1 if degree in x else 0)

    study_grouped = study_grouped.drop(columns=['fields_of_study'], axis=1)

    study_grouped = study_grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
        school_count=('school_name', 'count')), on=['user_id'], how='left')
    study_grouped = study_grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
        study_count=('fields_of_study', 'count')), on=['user_id'], how='left')
    study_grouped = study_grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
        school_nunique=('school_name', 'nunique')), on=['user_id'], how='left')
    study_grouped = study_grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
        study_nunique=('fields_of_study', 'nunique')), on=['user_id'], how='left')

    degree_grouped = degree_grouped.drop(columns=['degree'], axis=1)
    grouped = study_grouped.merge(degree_grouped, on = ['user_id'], how = 'left')

    return grouped

In [17]:
#df = load_education(path)

#df

100%|██████████| 10/10 [00:02<00:00,  4.24it/s]


,user_id,study_Computer Engineering,study_Electrical and Electronics Engineering,study_Computer Programming,study_Computer Science,"study_Business Administration and Management, General",study_Mechanical Engineering,study_Software Engineering,study_Management Information Systems,study_Industrial Engineering,...,degree_Bachelor's degree,degree_Master's degree,degree_Bachelor of Science,degree_High School,degree_Master of Science,degree_Associate's degree,degree_Bachelor of Engineering,degree_Doctor of Philosophy,degree_Master of Business Administration,degree_Mühendislik Fakültesi Mezunu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
2,2,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66267,66270,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
66268,66271,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
66269,66272,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
66270,66273,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
